In [7]:
import torch
import pandas as pd 
import torchvision
from torchvision import datasets, transforms
from torch import nn,optim
import torch.nn.functional as F
import numpy as np


training_data = pd.read_csv('file://localhost/C:/Users/Bryan/Documents/School/Y3/FYP/UJIndoorLoc/trainingData.csv')
test_data = pd.read_csv('file://localhost/C:/Users/Bryan/Documents/School/Y3/FYP/UJIndoorLoc/validationData.csv')

# Training data: 

In [8]:
# Get data from 1 floor only 

#building 1
b1 = training_data[training_data["BUILDINGID"] == 1]

#building 1 floor 1 
b1f1 = b1[b1["FLOOR"]==1]

#moving longitude and latitude into seperate label df 
label = b1f1[["LONGITUDE","LATITUDE"]]

#change long to positive value 
#label["LONGITUDE"] = label["LONGITUDE"].abs()




#remove unneccessary columns 
b1f1 = b1f1.drop(columns=['FLOOR','BUILDINGID','SPACEID','RELATIVEPOSITION','USERID','PHONEID','TIMESTAMP',"LONGITUDE","LATITUDE"])

# change all values 100 to -110 
b1f1 = b1f1.replace([100],-110)

#normalize all RSSI values between 0 to 1 
# (x - xmin)/(xmax - xmin)
b1f1 = b1f1.apply(lambda x: (x-x.min())/(x.max()-x.min()), axis=1)






#convert df to tensor object
b1f1_t = torch.tensor(b1f1.values)

label_t = torch.tensor(label.values)

#flatten 
b1f1_t = torch.flatten(b1f1_t,1)

label_t = torch.flatten(label_t,1)
print(label_t.shape)
print(b1f1_t.shape)
# print(label_t[0])
# print(b1f1_t[0])


train_data = [] 
for i in range(len(b1f1_t)):
    train_data.append([b1f1_t[i],label_t[i]])

trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=100)

print(len(trainloader))

torch.Size([1484, 2])
torch.Size([1484, 520])
15


# Test data


In [3]:
# test dataset stuff

b1test = test_data[test_data["BUILDINGID"] == 1]

b1f1test = b1test[b1test["FLOOR"]==1]

labeltest = b1f1test[["LONGITUDE","LATITUDE"]]

b1f1test = b1f1test.drop(columns=['FLOOR','BUILDINGID','SPACEID','RELATIVEPOSITION','USERID','PHONEID','TIMESTAMP',"LONGITUDE","LATITUDE"])

b1f1test = b1f1test.replace([100],-110)

b1f1test = b1f1test.apply(lambda x: (x-x.min())/(x.max()-x.min()), axis=1)

b1f1_test_t = torch.tensor(b1f1test.values)
label_test_t = torch.tensor(labeltest.values)

b1f1_test_t = torch.flatten(b1f1_test_t,1)
label_test_t = torch.flatten(label_test_t,1)
print(label_test_t)

print(label_test_t.shape)
print(b1f1_test_t.shape)

testloader = torch.utils.data.DataLoader([ [b1f1_test_t[i], label_test_t[i]] for i in range(len(label_test_t))], shuffle=True, batch_size=64)
print(len(testloader))


tensor([[  -7515.9168, 4864889.6629],
        [  -7568.9859, 4864876.0258],
        [  -7562.7171, 4864865.9577],
        [  -7543.7692, 4864923.9028],
        [  -7543.9604, 4864927.2163],
        [  -7516.4136, 4864937.6504],
        [  -7519.6728, 4864937.7624],
        [  -7505.6707, 4864864.8584],
        [  -7506.7627, 4864850.7019],
        [  -7511.4681, 4864852.6785],
        [  -7516.4423, 4864843.7234],
        [  -7535.1591, 4864937.0876],
        [  -7523.1986, 4864951.8533],
        [  -7519.3668, 4864949.7364],
        [  -7519.5859, 4864949.8245],
        [  -7510.2112, 4864949.2871],
        [  -7510.2112, 4864949.2871],
        [  -7528.6081, 4864959.5053],
        [  -7524.4868, 4864933.9102],
        [  -7526.7908, 4864929.9100],
        [  -7540.4677, 4864920.0487],
        [  -7537.0249, 4864918.2767],
        [  -7537.5785, 4864895.7401],
        [  -7437.6053, 4864862.8503],
        [  -7424.5911, 4864875.2112],
        [  -7423.6570, 4864876.0216],
        [  -

# Classifier

In [4]:
from torchsample.modules import ModuleTrainer 

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(520,500)
        self.fc2 = nn.Linear(500,500)
        self.fc3 = nn.Linear(500,500)
        self.fc4 = nn.Linear(500,2)
        
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self,x):
        x = x.view(x.shape[0],-1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.fc4(x)
        
        return x

In [5]:
from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):

      R = 6372.8 # this is in miles.  For Earth radius in kilometers use 6372.8 km

      dLat = radians(lat2 - lat1)
      dLon = radians(lon2 - lon1)
      lat1 = radians(lat1)
      lat2 = radians(lat2)

      a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
      c = 2*asin(sqrt(a))

      return R * c





In [24]:
import statistics

model = Classifier()



optimizer = optim.Adam(model.parameters(), lr=0.003, eps = 1e-9) 
#optimizer = optim.SGD(model.parameters(), lr = 0.003, weight_decay=0.09, momentum=0.9)

criterion = nn.MSELoss() #MSE
#criterion = torch.nn.L1Loss()

epochs = 500
train_losses, test_losses = [],[]

for e in range(epochs):
    predicted_list, label_list,dist = [],[],[]
    tot_train_loss = 0 
    for data, labels in trainloader:
        data = data.view(data.shape[0],-1)
        data = data.float()
        optimizer.zero_grad()
        output = model(data)
        #output = ((output - torch.mean(output))/torch.max(output)-torch.min(output))
        loss = torch.sqrt(criterion(output,labels.float())) #RMSE 
        #loss = criterion(output,labels.float())
        tot_train_loss+=loss.item()

        loss.backward()
        optimizer.step()
        

    else:
        tot_test_loss = 0 
        with torch.no_grad():
            
            for data, labels in testloader:
                data = data.view(data.shape[0],-1)
                data=data.float()
                test_output = model(data)
                loss = torch.sqrt(criterion(test_output,labels.float()))
                #loss = criterion(test_output,labels.float())
                tot_test_loss+=loss.item()
        model.train()
                
        train_loss = tot_train_loss/len(trainloader.dataset)
        test_loss = tot_test_loss/len(trainloader.dataset)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        
        #Add predicted and label value into 2 lists for distance calculation 
        for i in range(len(test_output)):
            predicted_list.append(test_output[i])
            label_list.append(labels[i])
        #print(predicted_list)
        #print("new")

        for j in range(len(predicted_list)):
   
            try:
                dist.append(haversine(predicted_list[j][1].float(),predicted_list[j][0].float(),label_list[j][1].float(),label_list[j][0].float()))
            except ValueError:
                dist.append(0)
        try:
            print("Epoch:{}..".format(e+1),
                 #"Training Loss:{:.3f}..".format(train_loss),
                 "Test Loss:{:.3f}..".format(test_loss),
                  "Min Dist Diff:{:.3f}..".format(min(dist)),
                  "Median Dist Diff:{:.3f}..".format(statistics.median(dist)),
                 "Avg Dist Diff:{:.3f}..".format(sum(dist)/len(dist)))
        except ValueError:
            print("error")
        
            


Epoch:1.. Test Loss:6952.431.. Min Dist Diff:3058.724.. Median Dist Diff:10041.983.. Avg Dist Diff:10138.507..
Epoch:2.. Test Loss:6907.819.. Min Dist Diff:3814.241.. Median Dist Diff:9896.610.. Avg Dist Diff:10499.495..
Epoch:3.. Test Loss:6557.864.. Min Dist Diff:753.148.. Median Dist Diff:10639.811.. Avg Dist Diff:10110.110..
Epoch:4.. Test Loss:5116.554.. Min Dist Diff:1513.092.. Median Dist Diff:7645.723.. Avg Dist Diff:7128.721..
Epoch:5.. Test Loss:2452.782.. Min Dist Diff:3208.991.. Median Dist Diff:12548.035.. Avg Dist Diff:10927.526..
Epoch:6.. Test Loss:2729.972.. Min Dist Diff:1207.725.. Median Dist Diff:10323.013.. Avg Dist Diff:9136.860..
Epoch:7.. Test Loss:2832.548.. Min Dist Diff:2340.562.. Median Dist Diff:9680.201.. Avg Dist Diff:9136.047..
Epoch:8.. Test Loss:2886.247.. Min Dist Diff:5915.068.. Median Dist Diff:10852.971.. Avg Dist Diff:10863.065..
Epoch:9.. Test Loss:2714.923.. Min Dist Diff:601.518.. Median Dist Diff:10904.137.. Avg Dist Diff:9643.957..
Epoch:10..

Epoch:75.. Test Loss:1676.570.. Min Dist Diff:3038.363.. Median Dist Diff:12028.213.. Avg Dist Diff:10411.758..
Epoch:76.. Test Loss:1760.096.. Min Dist Diff:2989.843.. Median Dist Diff:9925.942.. Avg Dist Diff:9983.023..
Epoch:77.. Test Loss:1713.196.. Min Dist Diff:4212.382.. Median Dist Diff:9534.780.. Avg Dist Diff:10315.328..
Epoch:78.. Test Loss:1682.619.. Min Dist Diff:3886.053.. Median Dist Diff:10290.188.. Avg Dist Diff:9947.978..
Epoch:79.. Test Loss:1739.592.. Min Dist Diff:2597.594.. Median Dist Diff:10615.837.. Avg Dist Diff:11707.335..
Epoch:80.. Test Loss:1724.271.. Min Dist Diff:4065.742.. Median Dist Diff:9083.723.. Avg Dist Diff:9939.948..
Epoch:81.. Test Loss:1568.414.. Min Dist Diff:2236.647.. Median Dist Diff:9299.850.. Avg Dist Diff:10492.035..
Epoch:82.. Test Loss:1723.345.. Min Dist Diff:2012.371.. Median Dist Diff:10834.467.. Avg Dist Diff:9321.832..
Epoch:83.. Test Loss:1463.200.. Min Dist Diff:556.132.. Median Dist Diff:10558.755.. Avg Dist Diff:9838.557..
Ep

Epoch:149.. Test Loss:741.355.. Min Dist Diff:55.613.. Median Dist Diff:9347.824.. Avg Dist Diff:8388.519..
Epoch:150.. Test Loss:868.078.. Min Dist Diff:1562.175.. Median Dist Diff:8978.170.. Avg Dist Diff:9851.269..
Epoch:151.. Test Loss:877.175.. Min Dist Diff:980.038.. Median Dist Diff:11487.093.. Avg Dist Diff:10272.328..
Epoch:152.. Test Loss:815.798.. Min Dist Diff:3472.257.. Median Dist Diff:9110.798.. Avg Dist Diff:10634.303..
Epoch:153.. Test Loss:819.178.. Min Dist Diff:3995.968.. Median Dist Diff:9303.159.. Avg Dist Diff:8756.439..
Epoch:154.. Test Loss:904.995.. Min Dist Diff:1956.274.. Median Dist Diff:9235.735.. Avg Dist Diff:9172.912..
Epoch:155.. Test Loss:839.434.. Min Dist Diff:2442.375.. Median Dist Diff:11984.007.. Avg Dist Diff:11035.139..
Epoch:156.. Test Loss:832.307.. Min Dist Diff:176.662.. Median Dist Diff:9596.193.. Avg Dist Diff:10807.781..
Epoch:157.. Test Loss:764.173.. Min Dist Diff:386.882.. Median Dist Diff:7952.683.. Avg Dist Diff:8636.441..
Epoch:158

Epoch:223.. Test Loss:677.305.. Min Dist Diff:3669.841.. Median Dist Diff:8676.263.. Avg Dist Diff:9842.149..
Epoch:224.. Test Loss:729.617.. Min Dist Diff:4353.570.. Median Dist Diff:10764.430.. Avg Dist Diff:10390.832..
Epoch:225.. Test Loss:683.486.. Min Dist Diff:2351.402.. Median Dist Diff:10601.538.. Avg Dist Diff:10529.186..
Epoch:226.. Test Loss:755.051.. Min Dist Diff:1992.818.. Median Dist Diff:11172.015.. Avg Dist Diff:11400.227..
Epoch:227.. Test Loss:756.584.. Min Dist Diff:2831.034.. Median Dist Diff:12199.973.. Avg Dist Diff:11484.056..
Epoch:228.. Test Loss:666.132.. Min Dist Diff:4014.047.. Median Dist Diff:10598.094.. Avg Dist Diff:10686.933..
Epoch:229.. Test Loss:751.886.. Min Dist Diff:6114.226.. Median Dist Diff:12921.364.. Avg Dist Diff:12120.875..
Epoch:230.. Test Loss:733.942.. Min Dist Diff:2590.968.. Median Dist Diff:10500.908.. Avg Dist Diff:10644.835..
Epoch:231.. Test Loss:680.563.. Min Dist Diff:3363.020.. Median Dist Diff:8684.571.. Avg Dist Diff:10148.4

Epoch:297.. Test Loss:662.748.. Min Dist Diff:5753.179.. Median Dist Diff:12014.201.. Avg Dist Diff:11091.305..
Epoch:298.. Test Loss:686.077.. Min Dist Diff:5360.141.. Median Dist Diff:8323.222.. Avg Dist Diff:8974.949..
Epoch:299.. Test Loss:589.565.. Min Dist Diff:3652.895.. Median Dist Diff:11548.715.. Avg Dist Diff:12016.248..
Epoch:300.. Test Loss:625.669.. Min Dist Diff:2149.915.. Median Dist Diff:8367.968.. Avg Dist Diff:9086.717..
Epoch:301.. Test Loss:645.634.. Min Dist Diff:1032.462.. Median Dist Diff:9453.228.. Avg Dist Diff:10075.595..
Epoch:302.. Test Loss:632.649.. Min Dist Diff:1865.080.. Median Dist Diff:7894.081.. Avg Dist Diff:8279.285..
Epoch:303.. Test Loss:664.808.. Min Dist Diff:3417.823.. Median Dist Diff:10080.338.. Avg Dist Diff:10260.996..
Epoch:304.. Test Loss:664.383.. Min Dist Diff:2662.200.. Median Dist Diff:8018.119.. Avg Dist Diff:7418.854..
Epoch:305.. Test Loss:771.714.. Min Dist Diff:2864.305.. Median Dist Diff:8630.364.. Avg Dist Diff:8747.471..
Epo

Epoch:372.. Test Loss:689.785.. Min Dist Diff:1792.198.. Median Dist Diff:9965.083.. Avg Dist Diff:9385.389..
Epoch:373.. Test Loss:665.180.. Min Dist Diff:1727.283.. Median Dist Diff:8379.106.. Avg Dist Diff:8504.398..
Epoch:374.. Test Loss:660.307.. Min Dist Diff:6185.729.. Median Dist Diff:13446.530.. Avg Dist Diff:12692.259..
Epoch:375.. Test Loss:713.008.. Min Dist Diff:5929.984.. Median Dist Diff:9672.435.. Avg Dist Diff:10353.879..
Epoch:376.. Test Loss:566.612.. Min Dist Diff:4270.279.. Median Dist Diff:11356.119.. Avg Dist Diff:10828.319..
Epoch:377.. Test Loss:642.561.. Min Dist Diff:1436.042.. Median Dist Diff:10856.778.. Avg Dist Diff:10828.944..
Epoch:378.. Test Loss:648.695.. Min Dist Diff:1113.552.. Median Dist Diff:13883.335.. Avg Dist Diff:11312.603..
Epoch:379.. Test Loss:679.278.. Min Dist Diff:1167.302.. Median Dist Diff:11691.817.. Avg Dist Diff:10225.393..
Epoch:380.. Test Loss:616.319.. Min Dist Diff:4406.073.. Median Dist Diff:10527.100.. Avg Dist Diff:10638.412

Epoch:446.. Test Loss:630.644.. Min Dist Diff:1029.504.. Median Dist Diff:8362.505.. Avg Dist Diff:8920.281..
Epoch:447.. Test Loss:704.700.. Min Dist Diff:1189.033.. Median Dist Diff:9636.024.. Avg Dist Diff:9797.696..
Epoch:448.. Test Loss:577.995.. Min Dist Diff:3311.360.. Median Dist Diff:12763.757.. Avg Dist Diff:11340.450..
Epoch:449.. Test Loss:633.348.. Min Dist Diff:1905.554.. Median Dist Diff:9389.718.. Avg Dist Diff:8334.651..
Epoch:450.. Test Loss:659.719.. Min Dist Diff:4508.332.. Median Dist Diff:10474.789.. Avg Dist Diff:10696.770..
Epoch:451.. Test Loss:652.441.. Min Dist Diff:1577.620.. Median Dist Diff:9685.391.. Avg Dist Diff:9295.768..
Epoch:452.. Test Loss:641.432.. Min Dist Diff:4221.976.. Median Dist Diff:13011.269.. Avg Dist Diff:12586.643..
Epoch:453.. Test Loss:634.918.. Min Dist Diff:1024.839.. Median Dist Diff:12729.818.. Avg Dist Diff:11681.963..
Epoch:454.. Test Loss:681.950.. Min Dist Diff:1669.798.. Median Dist Diff:10270.610.. Avg Dist Diff:10002.926..


In [16]:
import statistics
print("Average loss: {:.3f}".format(sum(test_losses)/len(test_losses)))
print("Median loss: {:.3f}".format(statistics.median(test_losses)))
print("")
print("Average distance: {:.3f}".format(sum(dist)/len(dist)))
print("Median distance: {:.3f}".format(statistics.median(dist)))



Average loss: 1852.854
Median loss: 1358.659

Average distance: 9958.711
Median distance: 9946.778


In [9]:
torch.save(model.state_dict(), 'checkpoint1.pth')

In [11]:
state_dict = torch.load('checkpoint1.pth')
print(state_dict.keys())

odict_keys(['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias', 'fc4.weight', 'fc4.bias'])


In [12]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [13]:
print(model)

Classifier(
  (fc1): Linear(in_features=520, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=500, bias=True)
  (fc4): Linear(in_features=500, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
